In [2]:
import networkx as nx
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib inline
plt.style.use('ggplot')
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, graclus
from torch_cluster import graclus_cluster

AttributeError: module 'numpy' has no attribute '__config__'

In [ ]:
df2011raw = pd.read_csv('E:\Github\MLN\FinalData/2011raw.csv')
dfgraph = pd.read_csv('E:\Github\MLN\FinalData/GraphProp.csv')
dfcensus = pd.read_csv('E:\Github\MLN\FinalData/census.csv')

In [ ]:
df2011raw.head()

In [ ]:
df2011raw.shape

In [ ]:
edge_index = torch.LongTensor(list(range(0,1223)))
weight = torch.tensor(df2011raw['Total migrants Persons'].tolist())
num_nodes = df2011raw.shape[0]

In [ ]:
dfgraph.head()

In [ ]:
dfcensus.head()

In [ ]:
for i in range(34):
    print('{:28}  {}'.format(dfcensus['State'].tolist()[i], dfgraph.Id.tolist()[i]))

In [ ]:
dfgraph['Pop Size'] = dfcensus['Population']

In [ ]:
dfgraph = dfgraph.drop(columns=['Label', 'timeset'])

In [ ]:
dfgraph.columns

In [ ]:
dffeatures = dfgraph[['weighted indegree', 'weighted outdegree','indegree', 'outdegree', 'Degree', 'pageranks','clustering', 'Pop Size']]
dffeatures[['Area','Density','DecadalGrowth']] = dfcensus[['Area','Density','DecadalGrowth']]

In [ ]:
dffeatures.head()

In [ ]:
decadalgrowth = []

for i in dffeatures['DecadalGrowth'].str[:-1].tolist():
    try:
        decadalgrowth.append(np.float(i))
    except:
        decadalgrowth.append(-1 * float(i[1:]))

In [ ]:
dffeatures['DecadalGrowth'] = decadalgrowth

In [ ]:
dffeatures.head()

In [ ]:
dffeatures.dtypes

In [ ]:
dffeatures['Area']  =  pd.to_numeric(dffeatures['Area'].str.replace(',',''))

In [ ]:
dffeatures['Density']  =  pd.to_numeric(dffeatures['Density'].str.replace(',',''))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dffeatures[dffeatures.columns] = scaler.fit_transform(dffeatures[dffeatures.columns])

In [ ]:
dffeatures.head()

In [ ]:
dffeatures.columns

In [ ]:
dffeatures = dffeatures.drop(columns=['indegree', 'outdegree'])

In [ ]:
dffeatures = dffeatures.drop(columns=['Degree'])

In [ ]:
dffeatures = dffeatures.drop(columns=['clustering'])

In [ ]:
dffeatures = dffeatures.drop(columns=['weighted indegree', 'weighted outdegree', 'pageranks',])

In [ ]:
len(dffeatures.columns)

In [ ]:
from collections import Counter
a = dfcensus['Literacyrate'].to_numpy()
sorted(Counter(np.floor(a/10).astype(int)).keys())

In [ ]:
gt = np.floor(a/20).astype(int)
gt

In [ ]:
t = pd.read_csv('E:/Github/MLN/FinalData/Top10InterState.csv')
set(t['Target'].unique()) - set(t['Source'].unique())

In [ ]:
naming ={}
for i in range(32):
    naming[t['Source'].unique()[i]] = i
naming['goa'] =32
naming['lakshadweep'] = 33
naming['sikkim']=34
for i in range(t.shape[0]):
    t['Source'].iloc[i] = naming[t['Source'].iloc[i]]
    t['Target'].iloc[i] = naming[t['Target'].iloc[i]]
a = t['Source']
b = t['Target']
g = nx.DiGraph()
lp2011=[]
for i,j,k in zip(a, b, t['Weight'].tolist()):
    if i!=j:
        lp2011.append((i,j,k))
g.add_weighted_edges_from(lp2011)

In [ ]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn.pytorch import GraphConv
from dgl import DGLGraph
import networkx as nx
import numpy as np
import os, time

In [ ]:
G = dgl.DGLGraph(g)

In [ ]:
# message phase: all nodes first compute and send out messages along out-going edges.
mp = fn.copy_u('h', 'm')
# reduce phase: all node then collect in-coming messages, aggregate them and update their own embedding.
rp = fn.sum('m', 'h')

In [ ]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        # linear layer
        self.linear = nn.Linear(in_feats, out_feats)
   
    def forward(self, g, inputs):
        # g is the graph and the inputs is the input node features
        # first perform linear transformation
        h = self.linear(inputs)
        # set the node features
        g.ndata['h'] = h
        # using DGL api
        # 1st arg:
        # message phase: all nodes first compute and send out messages along out-going edges.
        # 2nd arg:
        # reduce phase: all node then collect in-coming messages, aggregate them and update their own embedding.
        g.update_all(fn.copy_u('h', 'm'), fn.sum('m', 'h'))
        # get the result node features
        h = g.ndata.pop('h')
        return h

In [ ]:
class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        # Custom Linear Layer 1
        self.gcn1 = GraphConv(len(dffeatures.columns), 4)
        # Custom Linear Layer 2 
        self.gcn2 = GraphConv(4, 2)
    
    def forward(self, g, features):
        x = th.relu(self.gcn1(g, features))
        x = self.gcn2(g, x)
        return x

In [ ]:
# defining the GCN
net = GCN()
print(net)

In [ ]:
# classes = {
#     0:4,
#     1:3,
# }

In [ ]:
gt2 = []
for i in gt:
    if i == 4:
        gt2.append(0)
    else:
        gt2.append(1)

In [ ]:
inputs   =  torch.from_numpy(dffeatures.values).float()
labels   = th.tensor([0, 1])  
labelled = th.tensor([0, 33]) 

In [ ]:
# d = []
# for i in range(len(dffeatures.values)):
#     if i == 14 or i==33:
#         pass
#     else:
#         d.append(dffeatures.values[i])

In [ ]:
# inputs   =  torch.from_numpy(np.array(d)).float()
# labels   = th.tensor([0, 1])  
# labelled = th.tensor([0, 32]) 

In [ ]:
# to store NN o/p
scores = []
net = GCN()
# list to store time
t=[]
# optimizer
optimizer = th.optim.Adam(net.parameters(), lr=1e-1)
# loop through epocs
for epoch in range(1000):
    if epoch >=3:
        t0 = time.time()
        
    logits = net(G, inputs)
    scores.append(logits)
    
    logp = F.log_softmax(logits, 1)

    # computing loss for the given nodes
    loss = F.nll_loss(logp[labelled], labels)
    
    # taking max score from each node prediction 
    d, indices = th.max(logits, dim=1)
    print(indices)
    # checking against the ground truth
    acc=th.sum(th.IntTensor(gt2) == indices).float()/len(gt2)
    # backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch >=3:
        t.append(time.time() - t0)
#     if epoch %100 ==0:    
    print('Epoch %d | Loss: %.4f | Accuracy: %.4f | Time(s) %.4f'% (epoch, loss.item(), acc, np.mean(t)))


In [ ]:
df2011raw.head()

In [ ]:
top5 = df2011raw.groupby('Area Name')['Work/employment Persons'].apply(lambda grp: grp.nlargest(10)).index
top5data = []
for i in top5:
    top5data.append([df2011raw['Last residence'].loc[i[1]], i[0], df2011raw['Work/employment Persons'].loc[i[1]]])
t = pd.DataFrame(top5data, columns=['Source', 'Target','Weight'])

In [ ]:
t.head()

In [ ]:
set(t['Target'].unique()) - set(t['Source'].unique())

In [ ]:
naming ={}
for i in range(31):
    naming[t['Source'].unique()[i]] = i
naming['andaman & nicobar islands'] =31
naming['goa'] =32
naming['lakshadweep'] = 33
naming['sikkim']=34
for i in range(t.shape[0]):
    t['Source'].iloc[i] = naming[t['Source'].iloc[i]]
    t['Target'].iloc[i] = naming[t['Target'].iloc[i]]

In [ ]:
t.shape

In [ ]:
a = t['Source']
b = t['Target']
g = nx.DiGraph()
lp2011=[]
for i,j,k in zip(a, b, t['Weight'].tolist()):
    if i!=j:
        lp2011.append((i,j,k))
g.add_weighted_edges_from(lp2011)

In [ ]:
G = dgl.DGLGraph(g)

In [ ]:
# to store NN o/p
scores = []
net = GCN()
# list to store time
t=[]
# optimizer
optimizer = th.optim.Adam(net.parameters(), lr=1e-1)
# loop through epocs
for epoch in range(100):
    if epoch >=3:
        t0 = time.time()
        
    logits = net(G, inputs)
    scores.append(logits)
    
    logp = F.log_softmax(logits, 1)

    # computing loss for the given nodes
    loss = F.nll_loss(logp[labelled], labels)
    
    # taking max score from each node prediction 
    d, indices = th.max(logits, dim=1)
    print(indices)
    # checking against the ground truth
    acc=th.sum(th.IntTensor(gt2) == indices).float()/len(gt2)
    # backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch >=3:
        t.append(time.time() - t0)
#     if epoch %100 ==0:    
    print('Epoch %d | Loss: %.4f | Accuracy: %.4f | Time(s) %.4f'% (epoch, loss.item(), acc, np.mean(t)))


In [ ]:
df2011raw.head()

In [ ]:
top5 = df2011raw.groupby('Area Name')['Business Persons'].apply(lambda grp: grp.nlargest(10)).index
top5data = []
for i in top5:
    top5data.append([df2011raw['Last residence'].loc[i[1]], i[0], df2011raw['Business Persons'].loc[i[1]]])
t = pd.DataFrame(top5data, columns=['Source', 'Target','Weight'])

In [ ]:
set(t['Target'].unique()) - set(t['Source'].unique())

In [ ]:
len(t['Source'].unique())

In [ ]:
naming ={}
for i in range(28):
    naming[t['Source'].unique()[i]] = i
naming['andaman & nicobar islands'] =28
naming['arunachal pradesh'] =29
naming['dadra & nagar haveli'] =30
naming['daman & diu'] =31
naming['goa'] =32
naming['lakshadweep'] = 33
naming['sikkim']=34
for i in range(t.shape[0]):
    t['Source'].iloc[i] = naming[t['Source'].iloc[i]]
    t['Target'].iloc[i] = naming[t['Target'].iloc[i]]

In [ ]:
a = t['Source']
b = t['Target']
g = nx.DiGraph()
lp2011=[]
for i,j,k in zip(a, b, t['Weight'].tolist()):
    if i!=j:
        lp2011.append((i,j,k))
g.add_weighted_edges_from(lp2011)

In [ ]:
G = dgl.DGLGraph(g)

In [ ]:
# to store NN o/p
scores = []
net = GCN()
# list to store time
t=[]
# optimizer
optimizer = th.optim.Adam(net.parameters(), lr=1e-1)
# loop through epocs
for epoch in range(100):
    if epoch >=3:
        t0 = time.time()
        
    logits = net(G, inputs)
    scores.append(logits)
    
    logp = F.log_softmax(logits, 1)

    # computing loss for the given nodes
    loss = F.nll_loss(logp[labelled], labels)
    
    # taking max score from each node prediction 
    d, indices = th.max(logits, dim=1)
    print(indices)
    # checking against the ground truth
    acc=th.sum(th.IntTensor(gt2) == indices).float()/len(gt2)
    # backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch >=3:
        t.append(time.time() - t0)
#     if epoch %100 ==0:    
    print('Epoch %d | Loss: %.4f | Accuracy: %.4f | Time(s) %.4f'% (epoch, loss.item(), acc, np.mean(t)))


In [ ]:
rankgdp = pd.read_excel('E:\Github\MLN\FinalData\GDP2001.xlsx', sheet_name='Sheet2', header=None, names=['2001', '2011'])

In [ ]:
rankgdp = rankgdp.drop(columns=['2001'])

In [ ]:
gt = rankgdp['2011'].tolist()

In [ ]:
rankgdp[rankgdp['2011'] > 100000].index

In [ ]:
rankgdp[rankgdp['2011'] <= 100000].index

In [ ]:
gt3 =[]
for i in gt:
    if gt.index(i) in rankgdp[rankgdp['2011'] > 100000].index:
        gt3.append(1)
    else:
        gt3.append(0)

In [ ]:
np.array(gt3)

In [ ]:
gt

In [ ]:
len(gt3)

In [ ]:
dffeatures.columns